# KICKSTARTER: MODEL DEMONSTRATION ON "LIVE" DATA

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
kick_inter = pd.read_csv('../../data/02_intermediate/kick_inter.csv', 
                         parse_dates=['created_at', 'deadline', 
                                      'last_update_published_at', 
                                      'launched_at', 'state_changed_at'])

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11,15,16,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', None)

## PROCESS LIVE DATASET FOR MODEL TESTING

In [4]:
mask = ((kick_inter['state']=='successful') | (kick_inter['state']=='failed'))
latest_model_data = kick_inter.loc[mask]

In [5]:
print('Latest Datapoint for the Model: ',
      latest_model_data.sort_values(by=['deadline'], 
                                    ascending=False).head(1)['deadline'])

Latest Datapoint for the Model:  1   2019-07-18 03:55:00
Name: deadline, dtype: datetime64[ns]


In [6]:
kick_inter = kick_inter.loc[kick_inter['state'] == 'live']

In [7]:
kick_inter.drop(columns=['backers_count', 'converted_pledged_amount', 'country',
                         'currency_symbol', 'currency_trailing_code', 'current_currency', 
                         'friends', 'is_backing', 'is_starrable', 'is_starred', 'source_url',
                         'last_update_published_at', 'permissions', 'pledged', 
                         'slug', 'spotlight', 'staff_pick', 'static_usd_rate', 
                         'unread_messages_count','unseen_activity_count', 'usd_pledged', 
                         'usd_type', 'overall_category', 'creator_name', 'creator_slug', 'disable_communication'], 
                inplace=True)

In [8]:
kick_inter['blurb_word_count'] = kick_inter.blurb.str.split().str.len()

In [9]:
kick_inter['campaign_length'] = (kick_inter['deadline'] - kick_inter['launched_at']).dt.days

In [10]:
kick_inter['delta_created_launched'] = (kick_inter['launched_at'] - kick_inter['created_at']).dt.days

In [11]:
bool_series = pd.isnull(kick_inter["fx_rate"])  
kick_inter[bool_series].currency.value_counts()

USD    10366
EUR     2008
GBP     1572
CAD      891
AUD      538
MXN      291
SEK      161
HKD      155
CHF      109
NZD       97
SGD       97
DKK       85
NOK       65
JPY       10
Name: currency, dtype: int64

In [12]:
avg_fx_rate_10yr = {
    'EUR': 1.231885, 
    'MXN': 0.068639, 
    'USD': 1.000000, 
    'GBP': 1.473741, 
    'CAD': 0.869234, 
    'SEK': 0.131576, 
    'DKK': 0.165301, 
    'HKD': 0.128529, 
    'NOK': 0.14488,
    'AUD': 0.857369, 
    'SGD': 0.757244, 
    'CHF': 1.043232, 
    'NZD': 0.744532
}
currency = ['EUR', 'MXN', 'USD', 'GBP', 'CAD', 
            'SEK', 'DKK', 'HKD', 'NOK', 'AUD', 
            'SGD', 'CHF', 'NZD']

for cur in currency:
    null_exRatesEUR = ((kick_inter.fx_rate.isnull()) & (kick_inter.currency==cur))
    kick_inter.loc[null_exRatesEUR, 'fx_rate']=avg_fx_rate_10yr[cur]

In [13]:
kick_inter['goal_usd'] = kick_inter.goal * kick_inter.fx_rate

In [14]:
kick_inter.drop(columns=['created_at','fx_rate', 'launched_at', 
                         'state_changed_at', 'city', 'state_loc'], 
                inplace=True)

In [15]:
kick_inter.rename({'goal':'goal_original', 'state':'target', 
                   'sub_category': 'category', 'country_loc':'country'}, 
                  axis=1, inplace=True)

In [16]:
counts = kick_inter['id'].value_counts()
kick_inter[kick_inter['id'].isin(counts.index[counts == 2])]

,blurb,currency,deadline,goal_original,id,name,target,urls,category,country,blurb_word_count,campaign_length,delta_created_launched,goal_usd
6115,A Tarot Deck to empower EMPATHS (sensitive and...,USD,2019-08-12 00:56:54,10000.0,182072441,The Empath Tarot: a 92 Card Deck for Inner Hea...,live,"{""web"":{""project"":""https://www.kickstarter.com...",Playing Cards,US,17,60,17,10000.000000
6116,A Tarot Deck to empower EMPATHS (sensitive and...,USD,2019-08-12 00:56:54,10000.0,182072441,The EMPATH TAROT: for Inner Healing,live,"{""web"":{""project"":""https://www.kickstarter.com...",Playing Cards,US,17,60,17,10000.000000
6129,Biodegradable & All-Natural Mosquito Bait Stat...,USD,2019-08-11 22:30:08,10000.0,1498034193,BioSafe Mosquito Bait Stations,live,"{""web"":{""project"":""https://www.kickstarter.com...",Technology,US,6,60,49,8854.720000
6130,Biodegradable & All-Natural Mosquito Bait Stat...,USD,2019-08-11 22:30:08,10000.0,1498034193,BioSafe Mosquito Traps,live,"{""web"":{""project"":""https://www.kickstarter.com...",Technology,US,6,60,49,10000.000000
6134,Funding for pressing of the 4xLP and 2xCD vers...,EUR,2019-08-11 21:57:54,7075.0,591016927,"Hiroshimabend - ""J10C"" 4xLP and 2xCD funding. ...",live,"{""web"":{""project"":""https://www.kickstarter.com...",Electronic Music,AT,15,60,0,7951.913139
6135,Funding for pressing of the 4xLP and 2xCD vers...,EUR,2019-08-11 21:57:54,7075.0,591016927,"Hiroshimabend - ""J10C"" 4xLP and 2xCD funding v...",live,"{""web"":{""project"":""https://www.kickstarter.com...",Electronic Music,AT,15,60,0,7990.088919
6246,Be always ready to shoot instantly and carry y...,EUR,2019-07-22 08:00:00,5000.0,860541192,SPINN – The perfect camera carrying system,live,"{""web"":{""project"":""https://www.kickstarter.com...",Product Design,DE,14,40,18,5619.726600
6247,This camera plate fundamentally changes the wa...,EUR,2019-07-22 08:00:00,5000.0,860541192,SPINN CP.01 camera carrying system,live,"{""web"":{""project"":""https://www.kickstarter.com...",Product Design,DE,14,40,18,611715.823850
6339,18.5L Main compartment which can be expanded |...,NZD,2019-07-26 15:33:38,8000.0,1682004306,Arky: The world’s most functional backpack wit...,live,"{""api"":{""star"":""https://api.kickstarter.com/v1...",Product Design,NZ,24,45,13,5255.727280
6340,"One backpack for cycling, fitness, travel & ad...",NZD,2019-07-26 15:33:38,8000.0,1682004306,Arky:A 4-in-1 modular backpack with unlimited ...,live,"{""web"":{""project"":""https://www.kickstarter.com...",Product Design,NZ,23,45,13,5392.354480


In [17]:
kick_inter.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [18]:
# build a world regions dictionary 
world_regions = {
    'Northern America': ['US', 'CA', 'MX', 'GL', 'BM'], 
    'Northern Africa':['MA', 'TN', 'EG', 'LY', 'DZ', 'SD'], 
    'Eastern Africa':['KE', 'MG', 'MW', 'ET', 'UG', 'TZ', 'RW', 'MU', 'ZW', 'SO', 'SC', 'MZ', 'ZM', 'DJ', 
                      'SS'], 
    'Middle Africa':['CM', 'TD', 'CG', 'CD', 'GA', 'GQ', 'CF'], 
    'Southern Africa':['ZA', 'BW', 'SZ', 'LS'], 
    'Western Africa':['GH', 'NG', 'LR', 'SN', 'ML', 'SL', 'NE', 'BF', 'GN', 'GM', 'CI', 'CV', 'MR', 'BJ'], 
    'Caribbean':['HT', 'TT', 'PR', 'DO', 'JM', 'BS', 'CU', 'KY', 'CW', 'GP', 'SX', 'VI', 'BB', 
                 'AG', 'DM', 'LC', 'VC', 'TC', 'KN', 'GD'], 
    'Central America':['GT', 'SV','BZ', 'PA', 'HN', 'CR', 'NI'], 
    'South America':['CO', 'PE', 'VE', 'EC', 'AR', 'BR', 'CL', 'BO', 'SR', 'UY', 'PY', 'GY'], 
    'Central Asia':['KG', 'KZ', 'TJ'], 
    'Eastern Asia':['HK', 'JP', 'CN', 'MN', 'KR', 'MQ', 'TW', 'KP', 'MO'], 
    'South-eastern Asia':['ID', 'SG', 'VN', 'MY', 'TH', 'KH', 'PH', 'MM', 'LA', 'TL'], 
    'Southern Asia':['BD', 'IN', 'LK', 'NP', 'AF', 'PK', 'BT', 'IR', 'MV'], 
    'Western Asia':['IL', 'JO', 'AM', 'LB', 'AE', 'CY', 'PS', 'GE', 'YE', 'IQ', 'KW', 'AZ', 'SA', 
                    'SY', 'QA', 'BH', 'TR'], 
    'Eastern Europe':['PL', 'RO', 'UA', 'MD', 'HU', 'CZ', 'RU', 'BG', 'SK', 'BY', 'FO'], 
    'Northern Europe':['SE', 'GB', 'NO', 'IE', 'DK', 'FI', 'LT', 'IS', 'EE', 'SJ', 'LV', 'AX'], 
    'Southern Europe': ['ES', 'IT', 'HR', 'RS', 'PT', 'SI', 'GR', 'MT', 'BA', 'MK', 'XK', 'GI', 
                        'VA', 'MC', 'AL', 'ME'],
    'Western Europe':['FR', 'CH', 'DE', 'BE', 'NL', 'LU', 'AT', 'MC'], 
    'Australia and New Zealand':['AU', 'NZ'], 
    'Melanesia':['VU', 'PG', 'FJ', 'NC'], 
    'Micronesia':['FM', 'GU', 'KI'], 
    'Polynesia':['WS', 'TO', 'CK', 'PF', 'PN'], 
    'Antarctica': ['AQ']
}

In [19]:
# flip the dictionary to make each individual value in the value list a new key. 
countries_regions = {}
for key, val in world_regions.items():
    for i in range(len(val)):
        countries_regions[val[i]] = key

In [20]:
# let's make a for loop to assign the correct region to the correct country in our dataframe. 
countries = ['US', 'ES', 'FR', 'AU', 'SE', 'CH', 'CA', 'GB', 'IT', 'NO', 'HK',
       'KE', 'MX', 'IE', 'JP', 'DE', 'BE', 'NZ', 'NL', 'CN', 'CO', 'GT',
       'HR', 'PL', 'DK', 'MN', 'ID', 'LU', 'FI', 'KR', 'SG', 'GH', 'HT',
       'BD', 'RS', 'AT', 'TT', 'NG', 'PE', 'SV', 'PT', 'ZA', 'MG', 'VN',
       'MQ', 'IL', 'PR', 'RO', 'TW', 'UA', 'MD', 'JO', 'TR', 'CM', 'SI',
       'IN', 'VE', 'AM', 'EC', 'HU', 'AR', 'LK', 'MA', 'LB', 'BR', 'CL',
       'CZ', 'BO', 'GL', 'MY', 'TH', 'GR', 'MW', 'BZ', 'RU', 'AE', 'KH',
       'DO', 'CY', 'ET', 'UG', 'MT', 'PA', 'HN', 'BG', 'PH', 'LR', 'CR',
       'LT', 'TZ', 'SK', 'TN', 'WS', 'BA', 'IS', 'PS', 'BY', 'SN', 'MK',
       'KG', 'ML', 'SR', 'VU', 'PG', 'EE', 'NP', 'UY', 'GE', 'TD', 'AQ',
       'RW', 'CG', 'FM', 'SL', 'SJ', 'BW', 'TO', 'GU', 'YE', 'MM', 'AF',
       'CK', 'JM', 'CD', 'KZ', 'EG', 'MU', 'ZW', 'FJ', 'XK', 'PY', 'BS',
       'PK', 'CU', 'NI', 'KY', 'LV', 'CW', 'NE', 'GI', 'BF', 'SO', 'LY',
       'GP', 'SC', 'TJ', 'GN', 'GM', 'SX', 'MZ', 'VA', 'IQ', 'BT', 'VI',
       'KW', 'SZ', 'MC', 'CI', 'LA', 'AL', 'GY', 'BB', 'ZM', 'AG', 'DJ',
       'IR', 'DM', 'CV', 'NC', 'DZ', 'KP', 'FO', 'LS', 'TL', 'KI', 'PF',
       'MR', 'LC', 'VC', 'AX', 'MO', 'AZ', 'SA', 'SY', 'PN', 'GA', 'GQ',
       'TC', 'KN', 'QA', 'BJ', 'MV', 'SS', 'SD', 'ME', 'BH', 'GD', 'CF',
       'BM']

for country in countries:
    kick_inter.loc[kick_inter['country']==country, 'world_regions']=countries_regions[country]

In [21]:
kick_inter.reset_index(inplace=True, drop=True)

Let's filter this dataset down to deadlines that are between our latest model training deadline and today. We are doing this so that we can randomly choose campaigns to test our model on. 

In [22]:
latest_date = '2019-08-06'
earliest_date = '2019-07-19'

mask = ((kick_inter['deadline'] < latest_date) & (kick_inter['deadline'] > earliest_date))
live_prediction_df = kick_inter.loc[mask]
live_prediction_df.reset_index(inplace=True, drop=True)

In [23]:
live_prediction_df.to_pickle('../../data/03_processed/live_prediction_df.pkl')

## Let's make a prediction!

### Read in our final Adaboost model

In [24]:
filename = '../../data/04_models/final_model_ada.sav'
final_model_ada = pickle.load(open(filename, 'rb'))

In [25]:
final_model_ada

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=180, random_state=None)

### Prepare our dataset

In [26]:
live_prediction_df.head(3)

,blurb,currency,deadline,goal_original,id,name,target,urls,category,country,blurb_word_count,campaign_length,delta_created_launched,goal_usd,world_regions
0,"Stand tall. Be sweet. Wear a crown, if you wan...",USD,2019-08-02 02:24:42,500.0,1047879707,Prideapples!,live,"{""web"":{""project"":""https://www.kickstarter.com...",Accessories,US,15,15,3,500.0,Northern America
1,I'll be creating an enamel pin of Eleven from ...,USD,2019-07-29 22:50:05,370.0,592255111,Bad-ass Women Enamel Pins,live,"{""web"":{""project"":""https://www.kickstarter.com...",Art,US,15,12,0,370.0,Northern America
2,A story-focused futuristic sci-fi heist RPG wh...,USD,2019-07-23 00:00:00,100.0,874540810,Antimatter: A Cosmically Balanced RPG,live,"{""web"":{""project"":""https://www.kickstarter.com...",Tabletop Games,US,15,5,20,100.0,Northern America


In [27]:
X_feats = ['category', 'blurb_word_count', 'campaign_length',
           'delta_created_launched', 'goal_usd', 'world_regions']

In [28]:
X = pd.get_dummies(live_prediction_df[X_feats])

I've discovered that the testing dataset needs to have the same number of columns as the final set

In [29]:
final_model_features = ['blurb_word_count',
 'campaign_length',
 'delta_created_launched',
 'goal_usd',
 'category_3D Printing',
 'category_Academic',
 'category_Accessories',
 'category_Action',
 'category_Animals',
 'category_Animation',
 'category_Anthologies',
 'category_Apparel',
 'category_Apps',
 'category_Architecture',
 'category_Art',
 'category_Art Books',
 'category_Audio',
 'category_Bacon',
 'category_Blues',
 'category_Calendars',
 'category_Camera Equipment',
 'category_Candles',
 'category_Ceramics',
 "category_Children's Books",
 'category_Childrenswear',
 'category_Chiptune',
 'category_Civic Design',
 'category_Classical Music',
 'category_Comedy',
 'category_Comic Books',
 'category_Comics',
 'category_Community Gardens',
 'category_Conceptual Art',
 'category_Cookbooks',
 'category_Country & Folk',
 'category_Couture',
 'category_Crafts',
 'category_Crochet',
 'category_DIY',
 'category_DIY Electronics',
 'category_Dance',
 'category_Design',
 'category_Digital Art',
 'category_Documentary',
 'category_Drama',
 'category_Drinks',
 'category_Electronic Music',
 'category_Embroidery',
 'category_Events',
 'category_Experimental',
 'category_Fabrication Tools',
 'category_Faith',
 'category_Family',
 'category_Fantasy',
 "category_Farmer's Markets",
 'category_Farms',
 'category_Fashion',
 'category_Festivals',
 'category_Fiction',
 'category_Film & Video',
 'category_Fine Art',
 'category_Flight',
 'category_Food',
 'category_Food Trucks',
 'category_Footwear',
 'category_Gadgets',
 'category_Games',
 'category_Gaming Hardware',
 'category_Glass',
 'category_Graphic Design',
 'category_Graphic Novels',
 'category_Hardware',
 'category_Hip-Hop',
 'category_Horror',
 'category_Illustration',
 'category_Immersive',
 'category_Indie Rock',
 'category_Installations',
 'category_Interactive Design',
 'category_Jazz',
 'category_Jewelry',
 'category_Journalism',
 'category_Kids',
 'category_Knitting',
 'category_Latin',
 'category_Letterpress',
 'category_Literary Journals',
 'category_Literary Spaces',
 'category_Live Games',
 'category_Makerspaces',
 'category_Metal',
 'category_Mixed Media',
 'category_Mobile Games',
 'category_Movie Theaters',
 'category_Music',
 'category_Music Videos',
 'category_Musical',
 'category_Narrative Film',
 'category_Nature',
 'category_Nonfiction',
 'category_Painting',
 'category_People',
 'category_Performance Art',
 'category_Performances',
 'category_Periodicals',
 'category_Pet Fashion',
 'category_Photo',
 'category_Photobooks',
 'category_Photography',
 'category_Places',
 'category_Playing Cards',
 'category_Plays',
 'category_Poetry',
 'category_Pop',
 'category_Pottery',
 'category_Print',
 'category_Printing',
 'category_Product Design',
 'category_Public Art',
 'category_Publishing',
 'category_Punk',
 'category_Puzzles',
 'category_Quilts',
 'category_R&B',
 'category_Radio & Podcasts',
 'category_Ready-to-wear',
 'category_Residencies',
 'category_Restaurants',
 'category_Robots',
 'category_Rock',
 'category_Romance',
 'category_Science Fiction',
 'category_Sculpture',
 'category_Shorts',
 'category_Small Batch',
 'category_Software',
 'category_Sound',
 'category_Space Exploration',
 'category_Spaces',
 'category_Stationery',
 'category_Tabletop Games',
 'category_Taxidermy',
 'category_Technology',
 'category_Television',
 'category_Textiles',
 'category_Theater',
 'category_Thrillers',
 'category_Translations',
 'category_Typography',
 'category_Vegan',
 'category_Video',
 'category_Video Art',
 'category_Video Games',
 'category_Wearables',
 'category_Weaving',
 'category_Web',
 'category_Webcomics',
 'category_Webseries',
 'category_Woodworking',
 'category_Workshops',
 'category_World Music',
 'category_Young Adult',
 'category_Zines',
 'world_regions_Antarctica',
 'world_regions_Australia and New Zealand',
 'world_regions_Caribbean',
 'world_regions_Central America',
 'world_regions_Central Asia',
 'world_regions_Eastern Africa',
 'world_regions_Eastern Asia',
 'world_regions_Eastern Europe',
 'world_regions_Melanesia',
 'world_regions_Micronesia',
 'world_regions_Middle Africa',
 'world_regions_Northern Africa',
 'world_regions_Northern America',
 'world_regions_Northern Europe',
 'world_regions_Polynesia',
 'world_regions_South America',
 'world_regions_South-eastern Asia',
 'world_regions_Southern Africa',
 'world_regions_Southern Asia',
 'world_regions_Southern Europe',
 'world_regions_Western Africa',
 'world_regions_Western Asia',
 'world_regions_Western Europe']

prediction_set_features = list(X.columns)

In [30]:
def _locate_missing_features_model(X, final_model_features):
    l = []
    prediction_set_features = list(X.columns)
    final_model_feat = final_model_features
    if len(prediction_set_features) != len(final_model_feat):
        for origFeat in final_model_feat: 
            if origFeat not in prediction_set_features:
                l.append(origFeat)
        return l
    else: 
        print('All Features Match')

In [31]:
def _creat_empty_dataframe(column_list, num_rows):
    col_list = column_list
    num_cols = len(col_list)
    fill_with_zeroes = np.zeros(shape=(num_rows, num_cols))
    new_df = pd.DataFrame(fill_with_zeroes, columns=[col_list])
    return new_df

In [32]:
def prepare_missing_adaboost_columns(X_df, final_model_features, num_columns):
    X = X_df
    final_model_features = final_model_features
    missing_features = _locate_missing_features_model(X, final_model_features)
    missing_feat_df = _creat_empty_dataframe(missing_features, num_columns)
    return pd.concat([X_df, missing_feat_df], axis=1)
    

### Make our prediction 

In [33]:
X_final = prepare_missing_adaboost_columns(X, final_model_features, 1850)

In [34]:
predictions = final_model_ada.predict(X_final)

In [35]:
predictions

array([0, 0, 1, ..., 1, 0, 0], dtype=uint8)